# Multi-Warehouse Predictive Replenishment Demo

This notebook demonstrates the complete pipeline:
1. Generate synthetic data
2. Run ETL
3. Train forecasts
4. Run optimization
5. Simulate plan execution
6. Generate summary reports and visuals


In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Add src to path
sys.path.insert(0, str(Path('..').resolve()))

from src.data.synthetic_generator import SyntheticDataGenerator
from src.etl.etl import ETLPipeline
from src.forecast.forecast_manager import ForecastManager
from src.optimization.replenishment_optimizer import ReplenishmentOptimizer
from src.simulator.transfer_simulator import TransferSimulator

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


## Step 1: Generate Synthetic Data


In [ ]:
# Generate data
generator = SyntheticDataGenerator(seed=42)
start_date = datetime.now() - timedelta(days=180)

data = generator.generate_all(
    n_warehouses=5,
    n_skus=50,
    days=180,
    start_date=start_date
)

# Save to data/raw
raw_dir = Path('../data/raw')
raw_dir.mkdir(parents=True, exist_ok=True)
generator.save_to_csv(data, raw_dir)

print(f"Generated data:")
for name, df in data.items():
    print(f"  {name}: {len(df)} rows")


## Step 2: Run ETL Pipeline


In [ ]:
# Run ETL
pipeline = ETLPipeline()
processed_dir = Path('../data/processed')
pipeline.process(raw_dir, processed_dir)

print("ETL complete")


## Step 3: Train Forecasts


In [ ]:
# Train forecasts
forecasts_dir = Path('../data/forecasts')
manager = ForecastManager(processed_dir, forecasts_dir)
manager.train_models()
manager.save_models()

# Generate forecasts
start_date = datetime.now()
manager.generate_all_forecasts(start_date, horizon=14)

print("Forecasting complete")


## Step 4: Run Optimization


In [ ]:
# Load data for optimizer
warehouses = pd.read_csv(processed_dir / 'warehouses.csv')
skus = pd.read_csv(processed_dir / 'skus.csv')
fleet = pd.read_csv(raw_dir / 'fleet.csv')

# Create optimizer
optimizer = ReplenishmentOptimizer(
    processed_dir / 'inventory_snapshot.csv',
    forecasts_dir / 'forecasts.csv',
    warehouses,
    skus,
    fleet
)

# Optimize
plan = optimizer.optimize(horizon_days=14)

# Save plan
output_dir = Path('../output/reports')
output_dir.mkdir(parents=True, exist_ok=True)
plan.to_csv(output_dir / 'replenishment_plan_sample.csv', index=False)

print(f"Generated plan with {len(plan)} transfers")
print(f"Total cost: ${plan['estimated_cost'].sum():,.2f}")


## Step 5: Simulate Plan Execution


In [ ]:
# Run simulation
simulator = TransferSimulator(
    output_dir / 'replenishment_plan_sample.csv',
    warehouses,
    skus,
    fleet,
    seed=42
)

inventory = pd.read_csv(processed_dir / 'inventory_snapshot.csv')
forecasts = pd.read_csv(forecasts_dir / 'forecasts.csv')

sim_report = simulator.generate_report(inventory, forecasts)

# Save report
sim_report['transfers'].to_csv(output_dir / 'simulation_report_sample.csv', index=False)

print("Simulation Summary:")
for key, value in sim_report['summary'].items():
    print(f"  {key}: {value}")


## Step 6: Generate Visualizations


In [ ]:
# Create output visuals directory
visuals_dir = Path('../output/visuals')
visuals_dir.mkdir(parents=True, exist_ok=True)

# 1. Forecast example
sample_forecast = forecasts[
    (forecasts['warehouse_id'] == forecasts['warehouse_id'].iloc[0]) &
    (forecasts['sku_id'] == forecasts['sku_id'].iloc[0])
].copy()
sample_forecast['date'] = pd.to_datetime(sample_forecast['date'])

plt.figure(figsize=(12, 6))
plt.plot(sample_forecast['date'], sample_forecast['forecast'], label='Forecast', linewidth=2)
plt.fill_between(sample_forecast['date'], sample_forecast['lower'], sample_forecast['upper'], 
                 alpha=0.3, label='Prediction Interval')
plt.xlabel('Date')
plt.ylabel('Forecasted Demand')
plt.title(f"Demand Forecast: {sample_forecast['sku_id'].iloc[0]} at {sample_forecast['warehouse_id'].iloc[0]}")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(visuals_dir / 'forecast_example.png', dpi=150)
plt.close()

print("Saved forecast_example.png")


In [ ]:
# 2. Plan Gantt chart
if len(plan) > 0:
    plan['ship_date'] = pd.to_datetime(plan['ship_date'])
    plan['expected_arrival'] = pd.to_datetime(plan['expected_arrival'])
    
    fig, ax = plt.subplots(figsize=(14, max(6, len(plan) * 0.3)))
    
    for idx, row in plan.iterrows():
        y_pos = idx
        duration = (row['expected_arrival'] - row['ship_date']).days
        ax.barh(y_pos, duration, left=row['ship_date'], 
               label=f"{row['from_wh']} → {row['to_wh']}", alpha=0.7)
    
    ax.set_yticks(range(len(plan)))
    ax.set_yticklabels([f"{r['from_wh']} → {r['to_wh']} ({r['sku_id']})" for _, r in plan.iterrows()])
    ax.set_xlabel('Date')
    ax.set_title('Replenishment Plan Timeline (Gantt)')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(visuals_dir / 'plan_gantt.png', dpi=150)
    plt.close()
    
    print("Saved plan_gantt.png")


In [ ]:
# 3. Cost breakdown
if len(plan) > 0:
    cost_by_destination = plan.groupby('to_wh')['estimated_cost'].sum().sort_values(ascending=False)
    
    plt.figure(figsize=(10, 6))
    cost_by_destination.plot(kind='bar')
    plt.xlabel('Destination Warehouse')
    plt.ylabel('Total Cost ($)')
    plt.title('Cost Breakdown by Destination Warehouse')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.savefig(visuals_dir / 'cost_breakdown.png', dpi=150)
    plt.close()
    
    print("Saved cost_breakdown.png")
    
    # Summary statistics
    print("\nPlan Summary:")
    print(f"  Total transfers: {len(plan)}")
    print(f"  Total cost: ${plan['estimated_cost'].sum():,.2f}")
    print(f"  Avg cost per transfer: ${plan['estimated_cost'].mean():,.2f}")
    print(f"  Unique SKUs: {plan['sku_id'].nunique()}")
    print(f"  Unique warehouses: {plan['to_wh'].nunique()}")


## Summary

The complete pipeline has been executed:
- ✅ Synthetic data generated
- ✅ ETL pipeline completed
- ✅ Forecasts trained and generated
- ✅ Replenishment plan optimized
- ✅ Plan execution simulated
- ✅ Visualizations created

Check the `/output` directory for sample data, reports, and visuals.
